In [73]:
from fractions import Fraction
from array import array
    
    
def transposeMatrix(m):
    """ Transposes the matrix"""
    return list(map(list, zip(*m)))

def getMatrixMinor(m,i,j):
    return [row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]

#Fraction().limit_denominator()

def getMatrixDeternminant(m):
    #base case for 2x2 matrix
    if len(m) == 2:
        return Fraction(m[0][0]*m[1][1]-m[0][1]*m[1][0]).limit_denominator()

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1)**c)*m[0][c]*getMatrixDeternminant(getMatrixMinor(m,0,c))
    return Fraction(determinant).limit_denominator()

def getMatrixInverse(m):
    # Edge case 1: matrix 1x1
    if len(m) == 1:
        return [[Fraction(1/m[0][0]).limit_denominator()]]

    determinant = getMatrixDeternminant(m)
    # Edge case 2: 2x2 matrix
    if len(m) == 2:
        return [[Fraction(m[1][1]/determinant).limit_denominator(), Fraction(-1*m[0][1]/determinant).limit_denominator()],
                [Fraction(-1*m[1][0]/determinant).limit_denominator(), Fraction(m[0][0]/determinant).limit_denominator()]]

    #find matrix of cofactors
    cofactors = []
    for r in range(len(m)):
        cofactorRow = []
        for c in range(len(m)):
            minor = getMatrixMinor(m,r,c)
            cofactorRow.append(((-1)**(r+c)) * getMatrixDeternminant(minor))
        cofactors.append(cofactorRow)
    cofactors = transposeMatrix(cofactors)
    for r in range(len(cofactors)):
        for c in range(len(cofactors)):
            cofactors[r][c] = Fraction(cofactors[r][c]/determinant).limit_denominator()
    return cofactors

def initZeroListsOfLists(list_of_lists):
    """ Initializes list of list with the required dimensions"""
    res = []
    for i in list_of_lists:
        res.append([Fraction(0) for _ in i])
    return res

def answer(m):
    """ Calculates the probabilities of reaching the terminal states"""
    
    # Get number of states.
    no_states = len(m)
    
    # Edge case 0: empty matrix.
    if (no_states == 0):
        print("Input matrix is empty")
        return []
        
    # Edge case 1: 1d matrix - Test 4 passed.
    if (no_states == 1):
        print("Input matrix is 1d")
        return [1, 1] # 0th state is final state for sure;)
    
    # Calculate tmp variable - sums of rows
    row_sums = [sum(i)  for i in m]
    #print("row_sums=", row_sums)
 

    # Get absorbing states.
    absorbing_states = []
    not_absorbing_states = []
    # Warning - assuming that m is square matrix
    transition_matrix = initZeroListsOfLists(m)
    for i in range(no_states):
        # If there are no outputs.
        if (row_sums[i] == 0):
            absorbing_states.append(i)
            transition_matrix[i][i] =  Fraction(1)
        # Or all outputs lead to the same node (diagonal):
        elif (row_sums[i] == m[i][i]) :
            absorbing_states.append(i)
            transition_matrix[i][i] = Fraction(1)
        else:
            not_absorbing_states.append(i)
            transition_matrix[i] = [Fraction(el / row_sums[i]).limit_denominator() for el in m[i]]
    #print("absorbing states ({}) = {}".format(len(absorbing_states), absorbing_states))
    #print("not absorbing states ({}) = {}".format(len(not_absorbing_states), not_absorbing_states))
    #print("transition_matrix=",transition_matrix)
    
    # Edge case 2: no terminal states (task states clearly that this cannot happen, but just in case...)
    if (len(absorbing_states) == 0):
        print("There are no absorbing states!")
        return []
    # The task clearly states that it is an absorbing Markov chain.

    # Edge case 3: all states are terminal states - which means that there are no transitions!
    # Edge case 1 is a special case of this edge case.
    if (len(not_absorbing_states) == 0):
        print("All states are absorbing!")
        res = [1] # 0-th state is the one where we will always finish
        for _ in range(len(not_absorbing_states)-1):
            res.append(0)
        res.append(1) # denominator
        return res

    # Change absorbing transition matrix into "standard form".
    # Swap cols and rows using advanced indexing.
    #transition_matrix[:][:] = transition_matrix [:][absorbing_states + not_absorbing_states]
    #transition_matrix[:][:] = transition_matrix [absorbing_states + not_absorbing_states, :]  
    # Swap cols.
    new_states = absorbing_states + not_absorbing_states
    for i in range(len(new_states)):
        m[i] = transition_matrix[new_states[i]]
        
    # Swap rows.
    # Use simple trick with list-of-list transposition.
    transposed_m = transposeMatrix(m)
    for i in range(len(new_states)):
        m[i] = transposed_m[new_states[i]]
    # Reverse the transposition.
    transition_matrix = transposeMatrix(m)
    #print("P =\n",transition_matrix)

    # Get R submatrix - transitions from not absorbing to absorbing states.
    R = [sublist[:len(absorbing_states)] for sublist in transition_matrix[len(absorbing_states):]]  
    #print("R =\n",R)
    
    # Get Q submatrix - transitions from not absorbing to not absorbing states.
    Q = [sublist[len(absorbing_states):] for sublist in transition_matrix[len(absorbing_states):]]
    #print("Q =\n",Q)
        
    # Calculate the fundamental matrix F.
    #F = (np.eye(len(not_absorbing_states)) - Q).I
    eye = []
    for i in range(len(not_absorbing_states)):
        eye.append([1 if i == j else 0 for j in range(len(not_absorbing_states))])
    #print("eye =\n",eye)
    diff = []
    for i in range(len(not_absorbing_states)):
        diff.append( [a-b for a,b in zip(eye[i], Q[i])])
    #print("diff =\n",diff)
    
    F = getMatrixInverse(diff)
    #print("F =\n",F)
    
    # Finally, calculate the limiting matrix - we can skip that at all.
    #P_limit = np.concatenate([np.concatenate( [np.eye(len(absorbing_states)), 
    #                              np.zeros(shape=(len(absorbing_states), len(not_absorbing_states)))], axis=1),
    #                         np.concatenate( [F * R, 
    #                              np.zeros(shape=(len(not_absorbing_states), len(not_absorbing_states)))], axis=1)],
    #                         axis =0)
    #print("P limit =\n",P_limit)
    
    # Only FxR part is interesting.
    # FxR_limit = F * R
    FxR_limit = initZeroListsOfLists(R)
    transposed_R = transposeMatrix(R)
    for r in range(len(not_absorbing_states)):
        # For each output row.
        for c in range(len(absorbing_states)):
            # For each output col.
            FxR_limit[r][c] = sum([a*b for a,b in zip(F[r], transposed_R[c])])
    #print("FxR_limit =\n",FxR_limit)
    
    # Get probabilities of starting from state 0 to final.
    # As we already fixed the case of s0 being terminal, now we are sure that s0 is not terminal, 
    # thus it is related to the first vector of FxR part of limiting matrix. 
    absorbing_state_probabilities = FxR_limit[0]
    #print("absorbing_state_probabilities =\n", absorbing_state_probabilities)
    
    
    numerators = []
    denominators = []
    fractions = [ Fraction(prob).limit_denominator() for prob in absorbing_state_probabilities]
    #print("Fractions: {}".format(fractions))
    
    # Handle separatelly numerators and denominators.
    for frac in fractions:
        numerators.append(frac.numerator)
        denominators.append(frac.denominator)
    #print("numerators: {}".format(numerators))
    #print("denominators: {}".format(denominators))

    # Calculate factors
    max_den = max(denominators)
    factors = [max_den // den for den in denominators]
    #print("factors: {}".format(factors))
    
    # Bring to common denominator.
    final_numerators = [num * fac for num, fac in zip(numerators, factors)]
    #print("final_numerators: {}".format(final_numerators))
    
    # Sanity check
    if (sum(final_numerators) != max_den ):
        print("Error! Numerators do not sum to denominator!")

    # Format output
    output = []
    
    output = [int(el) for el in final_numerators]
    output.append(max_den)
    return output


if __name__ == "__main__":
    ore_trans_mat = [
      [0,1,0,0,0,1],  # s0, the initial state, goes to s1 and s5 with equal probability
      [4,0,0,3,2,0],  # s1 can become s0, s3, or s4, but with different probabilities
      [0,0,0,0,0,0],  # s2 is terminal, and unreachable (never observed in practice)
      [0,0,0,0,0,0],  # s3 is terminalnumerators
      [0,0,0,0,0,0],  # s4 is terminal 
      [0,0,0,0,0,0],  # s5 is terminal
    ]

    #ore_trans_mat = [
    #    [0, 0, 0, 0],
    #    [0, 0, 0, 0],
    #    [0, 0, 0, 0],
    #    [0, 0, 0, 0]
    #]
    
    #ore_trans_mat = [
    #    [1000, 2000, 3000, 4000],
    #    [0, 1000, 0, 0],
    #    [0, 0, 10001, 0],
    #    [0, 0, 0, 16000]
    #]
    
    
    #ore_trans_mat =   [[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

    #ore_trans_mat = [[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
    
    # Tricky cases!
    #ore_trans_mat =   [[], []]

    #ore_trans_mat =   [[0, 1,  0], [0, 1,  0],  [0, 1, 0]]

    #ore_trans_mat =   [[0,  2,  3,  4]]
    
    #ore_trans_mat =   [[0, 2], [1],  [0], [0, 0]]
    
    #ore_trans_mat =   [[1]]
    #ore_trans_mat =   [[0,  0],  [0, 1]]
    #ore_trans_mat = [[0,1,0,1], [1, 0, 0, 1], [0, 0, 0, 0], [0, 1, 1, 0]]
    
    #ore_trans_mat

    ore_trans_mat =   [[0, .3, .3, .4], 
                       [0, 0, 0, 0], 
                       [0, 0, 1, 0], 
                       [.8, .1, .1, 0]]
    
    
    ore_trans_mat =   [[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [.1, 0, .8, .1], 
                       [.1, .1, .4, .4]]
    
    print("ore_trans_mat=",ore_trans_mat)

    print("answer =",answer(ore_trans_mat))

    

ore_trans_mat= [[1, 0, 0, 0], [0, 1, 0, 0], [0.1, 0, 0.8, 0.1], [0.1, 0.1, 0.4, 0.4]]
answer = [7, 1, 8]
